In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '../LaurieOnTracking')
import Metrica_IO as mio
import Metrica_Viz as mviz
from itertools import groupby
import ast

In [2]:
result_matches = [] 

In [3]:
matches = ['metrica_1', 'metrica_2']
match = 'metrica_1'

In [4]:
# load event data from Metrica always to use reverse function
DATADIR = '../sample-data-master/data'
game_id = 1 # let's look at sample match 2

# read in the event data
events = mio.read_event_data(DATADIR,game_id)
if (match == 'metrica_1') or (match == 'metrica_2'):
    if match == 'metrica_1':
        game_id = 1 # let's look at sample match 1
    elif match == 'metrica_2':
        game_id = 2

    # read in the event data
    events = mio.read_event_data(DATADIR,game_id)

    # Bit of housekeeping: unit conversion from metric data units to meters
    events = mio.to_metric_coordinates(events)
    events.head()
    #### TRACKING DATA ####

    # READING IN TRACKING DATA
    tracking_home = mio.tracking_data(DATADIR,game_id,'Home')
    tracking_away = mio.tracking_data(DATADIR,game_id,'Away')

    # Convert positions from metrica units to meters 
    tracking_home = mio.to_metric_coordinates(tracking_home)
    tracking_away = mio.to_metric_coordinates(tracking_away)
    # reverse direction of play in the second half so that home team is always attacking from right->left
    tracking_home,tracking_away,events = mio.to_single_playing_direction(tracking_home,tracking_away,events)
    if match=='metrica_1':
        tracking_home, tracking_away, events= mio.change_playing_direction_both(tracking_home, tracking_away, events)
    #query features
    query_features = pd.read_csv('.././results' + '/metrica_' + str(game_id) +
                                     '_query_features.csv', sep=";")
    


Reading team: home
Reading team: away


In [5]:
# ball laying still--> set piece or outside
#when zero ball velocity or nan --> "Out"
query_features['close to ball'] = np.where((pd.isnull(query_features['ball velocity'])) |(query_features['ball velocity']== 0) | (pd.isnull(query_features['close to ball']))
                                           , "Out", query_features['close to ball'])
#
event = query_features['close to ball']
values = [event, event.shift(-1), event.shift(-2), event.shift(-3), event.shift(-4), event.shift(-5)]
sequence = np.all([v == values[0] for v in values], axis=0)
#[event[x]= np.where(sequence[x], event[x], event[x-1]) for x in range(1, len(event))]
possession = []
value = 'Out'
# loop through possession, is 5 in a row change,
for x in range(len(event)):
    if sequence[x]:
        value = event.iloc[x]
    elif value == 'Out': #changes when ball out or set piece
        value=event.iloc[x]
    possession.append(value)
    
query_features['possession'] = possession
#query_features = query_features.drop(columns=['closest to ball_home', 'closest to ball_away', 'close to ball'])
#query_features = query_features.drop(columns=['closest to ball_home', 'closest to ball_away', 'close to ball'])

In [6]:
# total goalkicks
if (match == 'metrica_1') or (match == 'metrica_2'):
    goalkicks = events[(events.Subtype == 'GOAL KICK') | (events.Subtype == 'GOAL KICK-INTERCEPTION')]
elif (match == 'dfb_1') or (match == 'dfb_2'):
    goalkicks = events[events.Subtype == 'GoalKick']
    
events.Subtype.unique()
#goalkicks
events[events['Start Frame'] > 71066].head(25)
#7716

,Team,Type,Subtype,Period,Start Frame,Start Time [s],End Frame,End Time [s],From,To,Start X,Start Y,End X,End Y
892,Home,SET PIECE,KICK OFF,2,72135,2885.40,72135,2885.40,Player9,NaN,NaN,NaN,NaN,NaN
893,Home,PASS,NaN,2,72135,2885.40,72160,2886.40,Player9,Player6,-1.06,-0.00,11.66,-1.36
894,Home,BALL LOST,INTERCEPTION,2,72189,2887.56,72237,2889.48,Player6,NaN,11.66,-0.00,-12.72,21.08
895,Away,RECOVERY,INTERCEPTION,2,72237,2889.48,72237,2889.48,Player20,NaN,-12.72,21.08,NaN,NaN
896,Away,PASS,NaN,2,72279,2891.16,72307,2892.28,Player20,Player18,-7.42,17.00,-1.06,11.56
897,Away,PASS,NaN,2,72315,2892.60,72358,2894.32,Player18,Player22,0.00,12.24,12.72,31.96
898,Away,PASS,NaN,2,72358,2894.32,72384,2895.36,Player22,Player23,12.72,31.96,18.02,20.40
899,Away,BALL LOST,INTERCEPTION,2,72415,2896.60,72425,2897.00,Player23,NaN,16.96,19.72,16.96,12.24
900,Away,RECOVERY,NaN,2,72443,2897.72,72443,2897.72,Player23,NaN,18.02,19.04,NaN,NaN
901,Away,PASS,NaN,2,72465,2898.60,72483,2899.32,Player23,Player22,18.02,21.08,26.50,32.64


In [7]:
query_features.reset_index()
query_features.index +=1
query_features.head(10)

,frame,ball x,ball y,ball velocity,inside,zone,centroid x_home,centroid y_home,centroid_zone_home,rest defence_home,...,centroid x_away,centroid y_away,centroid_zone_away,rest defence_away,width_away,length_away,occupancy map_away,team distance_away,close to ball,possession
1,1,-4.210850,-7.976468,NaN,True,8.0,10.094571,-5.163961,11.0,9.0,...,-2.246204,-1.267493,8.0,5.0,60.867956,31.771274,"[0, 1, 0, 0, 2, 4, 1, 2]",9.028938,Out,Out
2,11,-7.553878,-17.627436,28.560805,True,9.0,10.047931,-6.105054,11.0,10.0,...,-1.672288,-1.857080,8.0,3.0,61.266504,32.389466,"[0, 0, 0, 1, 3, 6, 0, 0]",10.760712,Away,Away
3,21,-7.182348,-20.427676,2.905701,True,9.0,10.163927,-7.137878,11.0,10.0,...,-0.973833,-2.569088,8.0,2.0,61.505456,33.046666,"[1, 0, 0, 0, 3, 6, 0, 0]",10.939565,Away,Away
4,31,-6.145138,-20.814868,2.765478,True,9.0,10.325661,-8.100731,11.0,10.0,...,-0.236465,-3.244396,8.0,3.0,61.649412,33.450950,"[1, 0, 0, 0, 3, 6, 0, 0]",10.681626,Away,Away
5,41,-4.389248,-20.411492,2.537699,True,9.0,10.513568,-8.882724,11.0,10.0,...,0.526809,-3.826360,11.0,4.0,61.650500,33.563098,"[0, 0, 1, 0, 3, 6, 0, 0]",9.918641,Away,Away
6,51,1.426230,-16.040588,20.190681,True,12.0,10.687895,-9.395410,11.0,8.0,...,1.272456,-4.240868,11.0,5.0,61.494848,33.277428,"[1, 0, 1, 0, 1, 5, 1, 1]",8.839663,Out,Away
7,61,5.179584,-13.019212,20.190681,True,12.0,10.807898,-9.567083,11.0,8.0,...,1.935274,-4.435932,11.0,6.0,61.229512,32.544226,"[0, 1, 0, 0, 1, 5, 2, 1]",8.296240,Away,Away
8,71,4.942462,-12.285152,1.199442,True,12.0,10.849757,-9.377771,11.0,8.0,...,2.491530,-4.366783,11.0,5.0,60.832120,31.559168,"[0, 1, 0, 1, 1, 4, 2, 1]",8.035096,Away,Away
9,81,3.466730,-7.746288,14.779685,True,11.0,10.857654,-8.845392,11.0,8.0,...,2.941309,-4.019970,11.0,5.0,60.324908,30.543794,"[0, 1, 0, 1, 2, 3, 1, 2]",8.537391,Home,Away
10,91,1.807194,-2.583728,14.779685,True,11.0,10.904093,-8.020688,11.0,8.0,...,3.342975,-3.428302,11.0,5.0,59.681900,29.589158,"[0, 1, 1, 1, 2, 2, 1, 2]",9.867063,Away,Away


In [8]:
query=query_features.copy()
goalkick_cond= []
#conditionA = [(query['possession'] == 'Out'), 50]
# goal kick Home
conditionA = [(query['ball velocity'] == 0) | (query['ball velocity'].isnull()), 50]
conditionB = [(query['ball velocity'] != 0) & \
                 (query.zone == 17) & \
                 (query['rest defence_home'] <= 1) & \
                  (query['rest defence_away'] == 10) & \
                  (query['possession'] != 'Away'), 1]
goalkick_cond.append([conditionA, conditionB])#
# goal kick Away
conditionA = [(query['ball velocity'] == 0) | (query['ball velocity'].isnull()), 50]
#conditionA = [(query['possession'] == 'Out'), 50]

conditionB = [(query['ball velocity'] != 0) & \
                 (query.zone == 2) & \
                 (query['rest defence_away'] <= 1) & \
                  (query['rest defence_home'] == 10) & \
                  (query['possession'] != 'Home'), 1]
goalkick_cond.append([conditionA, conditionB])#

In [9]:
def find_intervals(query_interval, condition, extra_time):
    """
    Method to find intervals that meet the condition
    
    params:
        query_interval: dataframe with feature values
        condition: list with true/false values for on query_interval for a certain condition
        extra:time: number of frames till the next condition

    returns:
        intervals: list with [start_frame, range()]
    """
    query_interval['cond'] = condition
    query_interval['phase'] = (query_interval['cond'] != query_interval['cond'].shift(1)).fillna(0).cumsum(skipna=False)
    phases= query_interval.groupby('phase').agg(
        Condition=('cond', 'first'),
        Start_frame=('frame', 'first'),
        End_frame=('frame', 'last')
        )
    phases = phases[phases.Condition]
    intervals = [[[], range(phases.Start_frame.iloc[x], phases.End_frame.iloc[x]+extra_time)] for x in range(len(phases))]
    return intervals

In [10]:
def find_intervals_custom(query_interval, condition, extra_time):
    """
    Method to find intervals that meet the condition
    
    params:
        query_interval: dataframe with feature values
        condition: list with true/false values for on query_interval for a certain condition
        extra:time: number of frames till the next condition

    returns:
        intervals: list with [start_frame, range()]
    """
    query_interval['cond'] = condition
    query_interval['phase'] = (query_interval['cond'] != query_interval['cond'].shift(1)).fillna(0).cumsum(skipna=False)
    # only first 5 seconds
    #query_interval = query_interval.groupby('phase')
    phases= query_interval.groupby('phase').agg(
        Condition=('cond', 'first'),
        Start_frame=('frame', 'first'),
        End_frame=('frame', 'last'),
        )
    phases['duration'] = (phases['End_frame'] - phases['Start_frame'])
    phases = phases[(phases.Condition) & (phases['duration'] > 20)]
    intervals = [[[], range(phases.Start_frame.iloc[x], phases.End_frame.iloc[x]+extra_time)] for x in range(len(phases))]

    return intervals

In [11]:
def check_conditions(query, conditions, framesize = 10):
    """
    Function to verify which conditions are indivually met
    
    params:
        query: Dataframe with feature items 
        conditions: listed with conditions in formation [["A", query.inside==False, int(sequence)], ...]
        framesize: the size of a frame, typically 10
    returns
        df query with additional column query that has letters for the conditions he meet
    """
    for condition in range(len(conditions)):
        if condition == 0:
                        query['query'] = np.where(conditions[condition][0], "L" + str(condition), "")
        else:
            query['query'] = query['query'] + np.where(conditions[condition][0], "L" + str(condition), "")

    return query


test = check_conditions(query_features, goalkick_cond[0], 10)
#test[(test.frame >=133263) & (test.frame<=133563)]

In [12]:
test[(test.frame >=22176)].head(25)

#test.iloc[8685:8692] # away home does not make sense according to possession close to ball
#test.iloc[10922]

,frame,ball x,ball y,ball velocity,inside,zone,centroid x_home,centroid y_home,centroid_zone_home,rest defence_home,...,centroid y_away,centroid_zone_away,rest defence_away,width_away,length_away,occupancy map_away,team distance_away,close to ball,possession,query
2219,22181,NaN,NaN,NaN,False,NaN,15.866091,5.255346,11.0,NaN,...,5.336973,11.0,NaN,39.652704,37.042760,NaN,4.519801,Out,Out,L0
2220,22191,NaN,NaN,NaN,False,NaN,15.390098,5.214940,11.0,NaN,...,5.184524,11.0,NaN,39.368532,36.928810,NaN,4.499462,Out,Out,L0
2221,22201,NaN,NaN,NaN,False,NaN,14.906239,5.193153,11.0,NaN,...,5.075561,11.0,NaN,39.098504,36.820054,NaN,4.401881,Out,Out,L0
2222,22211,NaN,NaN,NaN,False,NaN,14.421215,5.180145,11.0,NaN,...,4.993383,11.0,NaN,38.893484,36.792600,NaN,4.251381,Out,Out,L0
2223,22221,NaN,NaN,NaN,False,NaN,13.935184,5.174066,11.0,NaN,...,4.942832,11.0,NaN,38.744224,36.829382,NaN,4.110131,Out,Out,L0
2224,22231,NaN,NaN,NaN,False,NaN,13.445231,5.156889,11.0,NaN,...,4.936453,11.0,NaN,38.567288,36.910896,NaN,4.062359,Out,Out,L0
2225,22241,NaN,NaN,NaN,False,NaN,12.958754,5.124432,11.0,NaN,...,4.951345,11.0,NaN,38.355400,37.009158,NaN,4.072023,Out,Out,L0
2226,22251,NaN,NaN,NaN,False,NaN,12.472172,5.087774,11.0,NaN,...,4.972779,11.0,NaN,38.113728,37.059402,NaN,4.248045,Out,Out,L0
2227,22261,NaN,NaN,NaN,False,NaN,11.985537,5.051638,11.0,NaN,...,5.022990,11.0,NaN,37.810244,37.015942,NaN,4.352953,Out,Out,L0
2228,22271,NaN,NaN,NaN,False,NaN,11.514451,5.002801,11.0,NaN,...,5.075255,11.0,NaN,37.477384,36.893406,NaN,4.444692,Out,Out,L0


In [13]:
def find_intersections(intervals_combi, intervals_new):
    ''' Function to find which intervalt that intersect with each other
    
    params:
        intervals_combi: list with [[ranges[, range]
        intervals_new: list with [[ranges], range]
    returns:
        intersections: a list of ranges from intervalsB that intersect with intervalS
    ''' 
   
    intervalsA = [intervals_combi[x][1] for x in range(len(intervals_combi))]
    old_intervals = [intervals_combi[x][0] for x in range(len(intervals_combi))]
    intervalsB = [intervals_new[x][1] for x in range(len(intervals_new))]

    intersections= []
    
    i = j = 0
    valueA = np.nan
    valueB = np.nan
    while i < len(intervalsA) and j < len(intervalsB):
        # a before b, otherwise increase j
        if min(intervalsA[i]) > min(intervalsB[j]):
            j +=1
            continue
                  
        # Left bound for intersecting segment / biggest start          
        left = max(min(intervalsA[i]), min(intervalsB[j]))
        # Right bound / smallest end point
        right = min(max(intervalsA[i]), max(intervalsB[j]))
            
        if left <= right:
            # deal with two different intervals after for the same A
            if intervalsA[i] != valueA:
                old_interval = old_intervals[i]
                old_interval.append(intervalsA[i])
            if intervalsB[j] != valueB:
                intersections.append([old_interval, intervalsB[j]])
                valueA = intervalsA[i]
                valueB = intervalsB[j]
       
        # If i-th interval's right bound is  
        # smaller increment i else increment j 
        if max(intervalsA[i]) < max(intervalsB[j]):
            i += 1
        else:
            j += 1
        
    return intersections

In [14]:
def check_results(test, conditions):
    """ Functions that checks which episodes meet the conditions
    
    Params:
        test: DF with all features
        conditions: listed with conditions in formation [["A", query.inside==False], ...]
    returns:
        results: list with all episodes that meet the conditions
    """
    # Check conditions and put letter in query DF
    interval_combi = find_intervals_custom(test, conditions[0][0], conditions[0][1])
    
    for num_cond in range(1, len(conditions)):
        if num_cond==100:
            interval = find_intervals_custom(test, conditions[num_cond][0], conditions[num_cond][1])
        else:
            interval = find_intervals(test, conditions[num_cond][0], conditions[num_cond][1]) 
        interval_combi = find_intersections(interval_combi, interval)

    return interval_combi
        

# get all results and make episode of it
new_results = []
#for corner in cornerQ:
for expression in goalkick_cond:
    results = (check_results(query, expression))
    for item in results:
        try:
            start_time = min(item[1]) # item[0]
            end_time = max(item[1]) #- 51
            new_results.append(range(start_time-5*25, end_time+15*25))
        except: 
            continue


[[[range(7791, 8531)], range(8481, 8492)], [[range(12711, 13591)], range(13541, 13562)], [[range(21991, 22621)], range(22571, 22592)], [[range(32951, 33691)], range(33641, 33652)], [[range(35691, 36261)], range(36211, 36242)], [[range(69951, 70501)], range(70451, 70482)], [[range(84061, 84441)], range(84391, 84392)], [[range(86941, 87461)], range(87451, 87452)], [[range(97321, 98111)], range(98061, 98072)], [[range(101541, 102571)], range(102551, 102672)], [[range(120311, 120411)], range(120361, 120392)], [[range(121331, 121701)], range(121681, 121722)]]
[[[range(5961, 6731)], range(6681, 6702)], [[range(18311, 18601)], range(18551, 18562)], [[range(26641, 27291)], range(27241, 27252)], [[range(31241, 31591)], range(31541, 31552)], [[range(57741, 59101)], range(59051, 59062)], [[range(60181, 60841)], range(60791, 60802)], [[range(62851, 63281)], range(63231, 63242)], [[range(77081, 77551)], range(77501, 77572)], [[range(92141, 92621)], range(92571, 92582)], [[range(119481, 119561)], ra

In [17]:
# evaluate results
total = len(goalkicks) #len(corners) 
num_results = len(new_results)
tp = 0
for kicki in goalkicks['Start Frame']:
    if any([kicki in x for x in new_results]):
            tp+=1
     
fp = num_results - tp
fn = total - tp

precision = (tp) / (tp + fp)
recall = (tp) / (tp + fn)
fscore = (tp) / (tp + 0.5 * (fp + fn))

print(precision)
print(recall)
print(fscore)

result_matches.append([total, num_results, tp])

print(total)
print(num_results)
print(tp)

0.7272727272727273
0.7272727272727273
0.7272727272727273
22
22
16


In [18]:
result_matches

[[22, 22, 16]]

In [19]:
total = 0
num_results = 0
tp = 0
for match_result in result_matches:
    total += match_result[0]
    num_results += match_result[1]
    tp += match_result[2]

total+= 8 # unannotated cases
tp += 8

fp = num_results - tp
fn = total - tp

precision = (tp) / (tp + fp)
recall = (tp) / (tp + fn)
fscore = (tp) / (tp + 0.5 * (fp + fn))

print(total)
print(precision)
print(recall)
print(fscore)

30
1.0909090909090908
0.8
0.9230769230769231


In [20]:
STOP
DATADIR = '../../data_clean/results/goalkick_home/TQ_check/'
PLOTDIR = DATADIR
start_minute = 12
end_minute = 13


start_frame= 12491   - 5*25
end_frame = 12491 + 15*25
home = tracking_home[(tracking_home.index >=start_frame) & (tracking_home.index <= end_frame)]
away = tracking_away[(tracking_away.index >=start_frame) & (tracking_away.index <= end_frame)]
mviz.save_match_clip(home, away,PLOTDIR,fname='goalkick_homex', include_player_velocities=False)


NameError: name 'STOP' is not defined